In [ ]:
! pip install pandas simpledorff

In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import files

# 1. 上傳檔案
print("請上傳 Excel 檔案 (標註一致性分析_2026-01-07 (1).xlsx)：")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

def calculate_krippendorff(reliability_data):
    """計算 Nominal 尺度的 Krippendorff's Alpha"""
    data = np.array(reliability_data)
    # 移除整行皆空(三人都沒標)的情況
    data = data[:, ~pd.isna(data).all(axis=0)]

    # 提取唯一類別並排除 N/A (不分大小寫)
    valid_elements = [str(v).strip() for v in data[~pd.isna(data)] if str(v).strip().upper() != 'N/A']
    unique_values = sorted(list(set(valid_elements)))

    if len(unique_values) < 2:
        return 1.0, 0, 0, "類別不足"

    val_to_idx = {v: i for i, v in enumerate(unique_values)}
    m = len(unique_values)
    coincidence_matrix = np.zeros((m, m))

    for j in range(data.shape[1]):
        unit_vals = [str(v).strip() for v in data[:, j] if pd.notna(v) and str(v).strip().upper() != 'N/A']
        mu_j = len(unit_vals)
        if mu_j >= 2:
            for i1 in range(mu_j):
                for i2 in range(mu_j):
                    if i1 != i2:
                        v1, v2 = unit_vals[i1], unit_vals[i2]
                        coincidence_matrix[val_to_idx[v1], val_to_idx[v2]] += 1 / (mu_j - 1)

    n = np.sum(coincidence_matrix)
    if n == 0: return 1.0, 0, 0, "無效標註"

    row_sums = np.sum(coincidence_matrix, axis=1)
    do = np.sum(coincidence_matrix) - np.trace(coincidence_matrix)
    de = sum(row_sums[i] * row_sums[j] for i in range(m) for j in range(m) if i != j) / (n - 1)

    alpha_v = 1 - (do / de) if de != 0 else 1.0
    return alpha_v, do, de, "成功"

# 2. 處理流程
summary_list = []
dimensions = ['承諾狀態', '驗證時間', '證據狀態', '證據品質']

with open("krippendorff_analysis.log", "w", encoding="utf-8-sig") as log_f:
    log_f.write("Krippendorff's Alpha 詳細分週計算日誌\n" + "="*60 + "\n")

    xls = pd.ExcelFile(file_name)
    sheet_names = [s for s in xls.sheet_names if "組" in s and "統計" not in s]

    for sheet in sorted(sheet_names):
        df = pd.read_excel(xls, sheet_name=sheet)

        # 按「週數」與「標註類型」分組
        # 確保週數與標註類型沒有空值
        df['週數'] = df['週數'].fillna('未知週數')
        df['標註類型'] = df['標註類型'].fillna('未知類型')

        grouped = df.groupby(['週數', '標註類型'])

        for (week, b_type), group_data in grouped:
            log_f.write(f"\n【{sheet} | {week} | {b_type}】\n" + "-"*40 + "\n")

            for dim in dimensions:
                # 根據你提供的表頭：標註者1_承諾狀態 ...
                cols = [f'標註者{i}_{dim}' for i in range(1, 4)]
                # 檢查這些欄位是否真的存在
                cols = [c for c in cols if c in group_data.columns]

                if len(cols) < 2: continue

                reliability_data = group_data[cols].values.T.tolist()
                alpha_v, do, de, status = calculate_krippendorff(reliability_data)

                summary_list.append({
                    '組別': sheet, '週數': week, '標註類型': b_type,
                    '維度': dim, 'Alpha': round(alpha_v, 4),
                    'Do': round(do, 4), 'De': round(de, 4)
                })

                log_f.write(f"  [{dim:5s}] Alpha: {alpha_v:.4f} | Do: {do:.2f} | De: {de:.2f} ({status})\n")

        print(f"完成: {sheet}")

# 3. 下載結果
pd.DataFrame(summary_list).to_csv("Alpha_分週詳細報告.csv", index=False, encoding='utf-8-sig')
print("\n計算完成！請檢查下載的檔案。")
files.download("krippendorff_analysis.log")
files.download("Alpha_分週詳細報告.csv")

請上傳 Excel 檔案 (標註一致性分析_2026-01-07 (1).xlsx)：


Saving 標註一致性分析_2026-01-07 (1).xlsx to 標註一致性分析_2026-01-07 (1) (1).xlsx
完成: 組1
完成: 組2
完成: 組3
完成: 組4
完成: 組5
完成: 組6
完成: 組7

計算完成！請檢查下載的檔案。


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import numpy as np
import os
from google.colab import files

# 1. 上傳檔案
print("請上傳您的 Excel 檔案：")
uploaded = files.upload()
file_name = list(uploaded.keys())[0]

def calculate_with_detailed_matrix(reliability_data, dim_name, log_f):
    data = np.array(reliability_data)
    data = data[:, ~pd.isna(data).all(axis=0)] # 移除全空

    # 類別提取
    valid_elements = [str(v).strip() for v in data[~pd.isna(data)] if str(v).strip().upper() != 'N/A']
    unique_values = sorted(list(set(valid_elements)))

    if len(unique_values) < 2:
        log_f.write(f"   [{dim_name}] 類別不足 2 種，略過矩陣計算。\n")
        return 1.0, 0, 0

    val_to_idx = {v: i for i, v in enumerate(unique_values)}
    m = len(unique_values)
    matrix = np.zeros((m, m))

    # 填充符合矩陣 (Coincidence Matrix)
    for j in range(data.shape[1]):
        unit_vals = [str(v).strip() for v in data[:, j] if pd.notna(v) and str(v).strip().upper() != 'N/A']
        mu_j = len(unit_vals)
        if mu_j >= 2:
            for i1 in range(mu_j):
                for i2 in range(mu_j):
                    if i1 != i2:
                        v1, v2 = unit_vals[i1], unit_vals[i2]
                        matrix[val_to_idx[v1], val_to_idx[v2]] += 1 / (mu_j - 1)

    n = np.sum(matrix)
    if n == 0: return 1.0, 0, 0

    # 計算 Do 與 De
    row_sums = np.sum(matrix, axis=1)
    do = np.sum(matrix) - np.trace(matrix)
    de = sum(row_sums[i] * row_sums[j] for i in range(m) for j in range(m) if i != j) / (n - 1)
    alpha = 1 - (do / de) if de != 0 else 1.0

    # --- 寫入老師想看的「中間過程」 ---
    log_f.write(f"   >>> 維度: {dim_name}\n")
    log_f.write(f"       類別映射: {val_to_idx}\n")
    log_f.write(f"       符合矩陣 (Coincidence Matrix):\n")
    # 將矩陣格式化輸出
    matrix_str = ""
    for row in matrix:
        matrix_str += "           " + str(list(np.round(row, 2))) + "\n"
    log_f.write(matrix_str)
    log_f.write(f"       觀察衝突值 (Do) = Sum(非對角線) = {do:.4f}\n")
    log_f.write(f"       期望衝突值 (De) = {de:.4f}\n")
    log_f.write(f"       Alpha = 1 - ({do:.4f} / {de:.4f}) = {alpha:.4f}\n\n")

    return alpha, do, de

# 2. 執行並輸出
summary_list = []
dimensions = ['承諾狀態', '驗證時間', '證據狀態', '證據品質']
xls = pd.ExcelFile(file_name)
sheet_names = [s for s in xls.sheet_names if "組" in s and "統計" not in s]

with open("krippendorff_academic_report.log", "w", encoding="utf-8-sig") as log_f:
    log_f.write("Krippendorff's Alpha 學術級詳細計算日誌\n" + "="*60 + "\n")

    for sheet in sorted(sheet_names):
        df = pd.read_excel(xls, sheet_name=sheet)
        grouped = df.groupby(['週數', '標註類型'])

        for (week, b_type), g_data in grouped:
            log_f.write(f"【{sheet} | {week} | {b_type}】\n")
            for dim in dimensions:
                cols = [f'標註者{i}_{dim}' for i in range(1, 4) if f'標註者{i}_{dim}' in g_data.columns]
                if len(cols) < 2: continue
                alpha_v, do, de = calculate_with_detailed_matrix(g_data[cols].values.T.tolist(), dim, log_f)
                summary_list.append({'組別': sheet, '週數': week, '類型': b_type, '維度': dim, 'Alpha': alpha_v})

# 3. 匯出結果
pd.DataFrame(summary_list).to_csv("Alpha_Final_Detailed.csv", index=False)
files.download("krippendorff_academic_report.log")

請上傳您的 Excel 檔案：


Saving 標註一致性分析_2026-01-07 (1).xlsx to 標註一致性分析_2026-01-07 (1) (2).xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>